In [253]:
import numpy as np
import itertools

In [254]:
M = EuclideanSpace(3, 'M', start_index=1)
X.<x,y,z> = M.chart()
# e = M.vector_frame('e')
# de = M.coframes()[1] 

eX = X.frame()
dX = X.coframe()


In [255]:
# Declaration of 0-form (scalar field)
f = M.scalar_field(x^2)
f.display()

# Evaluation of 0-form
# p = M.point((1,2,3))
# f(p)

M → ℝ
(x, y, z) ↦ x^2

In [340]:
# Declaration 1-form
a = M.diff_form(1, {eX:[x*y*z, x*y*z, x*y*z]}, name='a')
a.display(X)
# a[X, :] = [x*y*z, 0, x*y*z]
# a.display(eX)

# evaluation 1-form
# a(eX[1])(p)

a = x*y*z dx + x*y*z dy + x*y*z dz

In [257]:
b = M.diff_form(2, name='b')
b[eX, 1, 2] = x*y*z #dx^dy
b[eX, 1, 3] = x*y^2 #dx^dz
b[eX, 2, 3] = z*x^2*y^3-2*x*y #dy^dz
b.display(X)


# evaluation 1-form
# p = M.point((1,2,3))
# b(eX[1], eX[2])(p)

b = x*y*z dx∧dy + x*y^2 dx∧dz + (x^2*y^3*z - 2*x*y) dy∧dz

In [258]:
def numerical_sampler_kform(M, k_form, point):
    """
    Sample numerical values of a k-form at a point
    :param M: manifold
    :param k_form: k-form to sample
    :param point: point at which to sample
    :return: numerical values of the k-form at the point as a tensor
    """
    
    # Check if the k-form is a 0-form (scalar field)
    if k_form.degree() == 0: 
        p = M.point(point)
        return np.array(float(k_form(p)))
    
    if k_form.degree() == 1:
        p = M.point(point)
        X.<x,y,z> = M.chart()
        eX = X.frame()
        return np.array([float(k_form(eX[i])(p)) for i in range(1,M.dimension()+1)])
    
    if k_form.degree() == 2:
        p = M.point(point)
        X.<x,y,z> = M.chart()
        eX = X.frame()
        return np.array([[float(k_form(eX[i], eX[j])(p)) for j in range(1,M.dimension()+1)] for i in range(1,M.dimension()+1)])
    
    if k_form.degree() == 3:
        p = M.point(point)
        X.<x,y,z> = M.chart()
        eX = X.frame()
        return np.array([[[float(k_form(eX[i], eX[j], eX[k])(p)) for k in range(1,M.dimension()+1)] for j in range(1,M.dimension()+1)] for i in range(1,M.dimension()+1)])

In [259]:
def sample_numerical_kform_neighborhood_val(M, k_form, point, epsilon=1e-5):
        form = k_form
        p = np.array(point)
        directions = [np.array(np.eye(3, dtype=int)[i]) for i in range(3)]
        values = {"0": [numerical_sampler_kform(M, form, p)],
                        "1": [numerical_sampler_kform(M, form, p - epsilon*directions[0]), numerical_sampler_kform(M, form, p + epsilon*directions[0])],
                        "2": [numerical_sampler_kform(M, form, p - epsilon*directions[1]), numerical_sampler_kform(M, form, p + epsilon*directions[1])],
                        "3": [numerical_sampler_kform(M, form, p - epsilon*directions[2]), numerical_sampler_kform(M, form, p + epsilon*directions[2])]}
        return values
    

In [260]:
def numerical_d(k_form_neighborhood_vals, e=1e-5):
    """
    Sample numerical values of the exterior derivative of a k-form at a point
    :param M: manifold
    :param k_form: k-form to sample
    :param point: point at which to sample
    :return: numerical values of the exterior derivative of the k-form at the point as a tensor
    """
    k = len(k_form_neighborhood_vals["0"][0].shape)
    
    # If the k-form is a 0-form (scalar field)
    if k == 0: 
        dw = np.zeros(3)
        for i in range(3):
            dw[i] = (1/(2*e))*(k_form_neighborhood_vals[str(i+1)][1] - k_form_neighborhood_vals[str(i+1)][0])
        return dw
    
    # If the k-form is a 1-form
    elif k == 1:
        dw = np.zeros((3,3))
        for i in range(3):
            for j in range(3):
                if i!=j:
                    dw[i,j] = (1/(2*e))*(k_form_neighborhood_vals[str(j+1)][0][i] +\
                                                k_form_neighborhood_vals[str(i+1)][1][j] -\
                                                k_form_neighborhood_vals[str(j+1)][1][i] -\
                                                k_form_neighborhood_vals[str(i+1)][0][j])
        
        return dw
    
    # If the k-form is a 2-form
    elif k == 2:
        dw = np.zeros((3,3,3))      
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    if i!=j and j!=k and i!=k:
                        dw[i,j,k] = (1/(2*e))*(+k_form_neighborhood_vals[str(i+1)][1][j,k] -\
                                                k_form_neighborhood_vals[str(i+1)][0][j,k] -\
                                                k_form_neighborhood_vals[str(j+1)][1][i,k] +\
                                                k_form_neighborhood_vals[str(j+1)][0][i,k] +\
                                                k_form_neighborhood_vals[str(k+1)][1][i,j] -\
                                                k_form_neighborhood_vals[str(k+1)][0][i,j])
        return dw
    
    elif k >= 3:
        return np.zeros((3,)*4)
        
        
        

In [320]:
def count_inversions(arr):
    """Count inversions in O(n log n) time via merge sort"""
    def merge_sort(lst):
        if len(lst) <= 1:
            return lst, 0
        mid = len(lst) // 2
        left, inv_l = merge_sort(lst[:mid])
        right, inv_r = merge_sort(lst[mid:])
        merged = []
        i = j = inv_m = 0
        while i < len(left) and j < len(right):
            if left[i] <= right[j]:
                merged.append(left[i])
                i += 1
            else:
                merged.append(right[j])
                inv_m += len(left) - i  # key line
                j += 1
        merged += left[i:]
        merged += right[j:]
        return merged, inv_l + inv_r + inv_m
    _, count = merge_sort(arr)
    return count

def perm_sign(p):
    """Fast sign using inversion count"""
    return 1 if count_inversions(p) % 2 == 0 else -1

def numerical_star(k_form_p_vals):
    k = len(k_form_p_vals.shape)
    
    if k == 0: 
        star = np.zeros((3,3,3))
        perms = list(itertools.permutations([0, 1, 2]))
        for p in perms:
            sign = perm_sign(p)
            star[p[0], p[1], p[2]] = sign * k_form_p_vals
        return star
    
    elif k == 1:
        star = np.zeros((3,3))
        perms = list(itertools.combinations([0, 1, 2], 2))
        
        for p in perms:
            star[p[0], p[1]] = ((-1)**(1+p[0]+p[1]))*k_form_p_vals[list(set([0,1,2]) - set([p[0],p[1]]))[0]]
        star = star - star.transpose()
        return star
    
    elif k == 2:
        star = np.zeros(3)
        
        for p in range(3):
            star[p] = ((-1)**p)*(k_form_p_vals[list(set([0,1,2]) - set([p]))[0], list(set([0,1,2]) - set([p]))[1]])
        return star
    
    elif k == 3:
        star = k_form_p_vals[0,1,2]
        return np.array(star)
    
    else:
        assert False, "Not defined."
        

In [264]:
# computing d^2
p = (1,2,3)
form = a
initial_f = sample_numerical_kform_neighborhood_val(M, form, p)
directions = [np.array(np.eye(3, dtype=int)[i]) for i in range(3)]
epsilon = 1e-5

dfs = {
    "0": [numerical_d(initial_f)],
    "1": [numerical_d(sample_numerical_kform_neighborhood_val(M, form, p - epsilon*directions[0])), numerical_d(sample_numerical_kform_neighborhood_val(M, form, p + epsilon*directions[0]))],
    "2": [numerical_d(sample_numerical_kform_neighborhood_val(M, form, p - epsilon*directions[1])), numerical_d(sample_numerical_kform_neighborhood_val(M, form, p + epsilon*directions[1]))],
    "3": [numerical_d(sample_numerical_kform_neighborhood_val(M, form, p - epsilon*directions[2])), numerical_d(sample_numerical_kform_neighborhood_val(M, form, p + epsilon*directions[2]))]
}

np.round(numerical_d(dfs))

array([[[ 0.,  0.,  0.],
        [ 0.,  0., -0.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [-0.,  0.,  0.]],

       [[ 0., -0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]])

In [342]:
# d*d* at any point
def d_star_d_star_at_p(point, form):
    # TODO: remove the need of sampler
    directions = [np.array(np.eye(3, dtype=int)[i]) for i in range(3)]
    epsilon = 1e-5

    d_star_at_p_neighborhood = {
        "0": [numerical_d({k: [numerical_star(arr) for arr in v] for k, v in sample_numerical_kform_neighborhood_val(M, form, point).items()})],
        "1": [numerical_d({k: [numerical_star(arr) for arr in v] for k, v in sample_numerical_kform_neighborhood_val(M, form, point - epsilon*directions[0]).items()}), numerical_d({k: [numerical_star(arr) for arr in v] for k, v in sample_numerical_kform_neighborhood_val(M, form, point + epsilon*directions[0]).items()})],
        "2": [numerical_d({k: [numerical_star(arr) for arr in v] for k, v in sample_numerical_kform_neighborhood_val(M, form, point - epsilon*directions[1]).items()}), numerical_d({k: [numerical_star(arr) for arr in v] for k, v in sample_numerical_kform_neighborhood_val(M, form, point + epsilon*directions[1]).items()})],
        "3": [numerical_d({k: [numerical_star(arr) for arr in v] for k, v in sample_numerical_kform_neighborhood_val(M, form, point - epsilon*directions[2]).items()}), numerical_d({k: [numerical_star(arr) for arr in v] for k, v in sample_numerical_kform_neighborhood_val(M, form, point + epsilon*directions[2]).items()})]
    }

    d_star_d_star_at_p = numerical_d({k: [numerical_star(arr) for arr in v] for k, v in d_star_at_p_neighborhood.items()})
    return d_star_d_star_at_p

In [343]:
# *d*d at any point
def star_d_star_d_at_p(point, form):
    # TODO: remove the need of sampler
    star_d_at_neighborhood =  {
        "0": [numerical_star(numerical_d(sample_numerical_kform_neighborhood_val(M, form, point)))],
        "1": [numerical_star(numerical_d(sample_numerical_kform_neighborhood_val(M, form, point - epsilon*directions[0]))), numerical_star(numerical_d(sample_numerical_kform_neighborhood_val(M, form, point + epsilon*directions[0])))],
        "2": [numerical_star(numerical_d(sample_numerical_kform_neighborhood_val(M, form, point - epsilon*directions[1]))), numerical_star(numerical_d(sample_numerical_kform_neighborhood_val(M, form, point + epsilon*directions[1])))],
        "3": [numerical_star(numerical_d(sample_numerical_kform_neighborhood_val(M, form, point - epsilon*directions[2]))), numerical_star(numerical_d(sample_numerical_kform_neighborhood_val(M, form, point + epsilon*directions[2])))]
    }

    star_d_star_d_at_p = numerical_star(numerical_d(star_d_at_neighborhood))
    return star_d_star_d_at_p

In [344]:
# defining Hodge Laplacian
def Hodge_Laplacian_at_p(point, form):
    return d_star_d_star_at_p(point, form) + star_d_star_d_at_p(point, form)

In [345]:
(a.exterior_derivative().hodge_dual().exterior_derivative().hodge_dual()).display(X)

*d*da = (y + z) dx + (x + z) dy + (x + y) dz